Importações

In [2]:
import time
import scipy.linalg as sp
import numpy as np
import sympy as sym
from math import *


TODO: arrumar método de broyden
      criar método para calcular aproximação de função por diferenças finitas
      criar método para calcular jacobiana por diferenças finitas
      usar para newton no segundo item
      usar para a primeira jacobiana de broyden no segundo item
      

## ITEM 1

In [22]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
f1 = x1*x2 +x1 -3*x5
f2 = 2*x1*x2 +x1 +3*R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 -R[0]*x5
f3 = 2*x2*(x3**2) +R[-4]*x2*x3 + 2*R[-6]*(x3**2) +R[-5]*x3 -8*x5
f4 = R[-2]*x2*x4 +2*(x4**2) -4*R[0]*x5
f5 = x1*x2 +x1 +R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 +R[-6]*(x3**2) +R[-5]*x3 +(x4**2) -1
    
F = np.transpose(np.array([f1,f2,f3,f4,f5]))

In [23]:
## calculate values of F under a vector x
def calcF(F, x):
    Fx = []
    for f in F:
        Fx.append(f.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
    return np.array(Fx)

## define jacobian matrix
def J(F):
    Jmatrix = []
    for f in F:
        Jmatrix.append([f.diff(x1), f.diff(x2), f.diff(x3), f.diff(x4), f.diff(x5)])
        
    return np.array(Jmatrix)    


## calculate values of J under a vector x
def calcJ(J, x):
    Jx = []
    i = 0
    for linha in J:
        Jx.append([])
        for j in linha:
            Jx[i].append(j.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
        i += 1
        
        
    return np.array(Jx)

## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.



### Método de Newton

In [24]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    k = 1
    print("Começando com Xo = ", x)
    while k <= maxIterations:
        Fx = [-x for x in calcF(F,x)]
        Jx = calcJ(J(F), x)
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(Fx, np.float64))
        x = x + y
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            break
        else:
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            k += 1
        
    return output

In [33]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
print(methodNewton(n, Xo, tolerance, maxIterations, F))
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")


Começando com Xo =  [10 10 10 10 10]
1  Iteração x= [0.01065204 9.99922036 5.00102591 5.03641579 0.03645869] Norma:  15.769785630785213
2  Iteração x= [ 0.01001153 10.01012371  2.49920776  2.59112275  0.03674507] Norma:  3.498381268945265
3  Iteração x= [9.98373148e-03 1.00839478e+01 1.24338306e+00 1.43787284e+00
 3.68870704e-02] Norma:  1.7066139146402113
4  Iteração x= [9.68920699e-03 1.04317749e+01 6.06359646e-01 9.75892034e-01
 3.69557589e-02] Norma:  0.8603539169025427
5  Iteração x= [8.50591251e-03 1.18477258e+01 2.74174277e-01 8.66848855e-01
 3.69857062e-02] Norma:  1.4584772627650562
6  Iteração x= [5.41646124e-03 1.67133282e+01 1.03992448e-01 8.60022364e-01
 3.69918658e-02] Norma:  4.868583458334746
7  Iteração x= [2.92103470e-03 2.76408171e+01 6.02819450e-02 8.59724155e-01
 3.69765221e-02] Norma:  10.927576589262472
8  Iteração x= [3.45584277e-03 3.17170310e+01 6.96040198e-02 8.59516377e-01
 3.69616409e-02] Norma:  4.07622454194743
9  Iteração x= [3.43022114e-03 3.13294698e+0

### Método de Broyden

In [31]:
# define method
def methodBroyden(n, x, tolerance, maxIterations, F): # n is the number of equations and unknowns
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    print("Começando com Xo = ", x)
    Ao = calcJ(J(F), x)
    v = calcF(F, x)
    A = np.linalg.inv(np.array(Ao, dtype=np.float64))
    s = np.dot(-A,v)
    x = x + s
    k = 2
    
    print("X1 = ", x)
    while k <= maxIterations:
        w = v
        v = calcF(F, x)
        y = v - w
        z = -np.dot(A,y)
        p = np.dot(-np.transpose(s),z)
        ut = np.dot(np.transpose(s),A)
        A = A + (np.outer((s+z),ut))/p
        s = -np.dot(A,v)
        x = x + s
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(np.array(s, np.float64), 2) < tolerance:
            print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
            output = x
            break
        print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
        k += 1
        
    return output
             
    

In [34]:
# solve with Broyden's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(np.array(x)) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
print(methodBroyden(n, Xo, tolerance, maxIterations, F))
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [10 10 10 10 10]
X1 =  [0.0106520416154137 9.99922036085834 5.00102590899330 5.03641579334684
 0.0364586887839735]
2  Iteração x= [0.00930920994776896 9.99992314923572 3.68548669725861 3.74021496588458
 0.0366095664539330] Norma:  1951.5790985841434
3  Iteração x= [0.00812409975971151 10.0028495599410 2.12982498189956 2.22119667752175
 0.0367878041371119] Norma:  290.1492702297571
4  Iteração x= [0.00741548470735665 10.0071664428273 1.36874831487401 1.49585096488847
 0.0368748606960816] Norma:  229.7381311533295
5  Iteração x= [0.00692512323934356 10.0133588765681 0.875839559672884 1.04643125912261
 0.0369311110668205] Norma:  67.65373576346511
6  Iteração x= [0.00665183257170165 10.0202955343292 0.612066692438099 0.827619518503195
 0.0369610835524859] Norma:  28.137157616273615
7  Iteração x= [0.00652289985436536 10.0272607578088 0.484859773836035 0.743324571897385
 0.0369754028649233] Norma:  9.990048017875601
8  Iteração x= [0.00647631800395310 10.0334117223175 0

### Comparação dos resultados

Ambos os métodos são feitos para inicair de um ponto que já tenha uma aproximação relativamente boa. O método de Newton apresentou uma boa convergência, enquanto o de Broyden não convergiu. De fato, é possível perceber que em outras implementações do método de Broyden, com o ponto inicial dado para as funções dadas, a função não converge.

## ITEM 2

In [86]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
def f1(x):
    return (x[0]*x[1]) + x[0] -(3*x[4])

def f2(x):
    global R
    return 2*x[0]*x[1] +x[0] +3*R[-1]*(x[1]**2) +x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] -R[0]*x[4]

def f3(x):
    global R
    return 2*x[1]*(x[2]**2) +R[-4]*x[1]*x[2] + 2*R[-6]*(x[2]**2) +R[-5]*x[2] -8*x[4]

def f4(x):
    global R
    return R[-2]*x[1]*x[3] +2*(x[3]**2) -4*R[0]*x[4]

def f5(x):
    global R
    return x[0]*x[1] + x[0] +R[-1]*(x[1]**2) + x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] +R[-6]*(x[2]**2) +R[-5]*x[2] +(x[3]**2) -1

Fmatrix = [f1, f2, f3, f4, f5]
    
def F(x):
    return np.transpose(np.array([f1(x),f2(x),f3(x),f4(x),f5(x)], np.float64))


In [81]:
# parcial derivatives by finite differences - FIRST ORDER
def df1(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/h
    
    return df_xk


# parcial derivatives by finite differences - SECOND ORDER
def df2(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x0[k] -= h
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/(2*h)
    
    return df_xk


## define jacobian matrix
def J(Fmatrix, x, order):
    h = .01
    Jmatrix = []
    
    if order == 1:
        for f in Fmatrix:
            Jmatrix.append([df1(f, h, x, 0), df1(f, h, x, 1), df1(f, h, x, 2), df1(f, h, x, 3), df1(f, h, x, 4)])
    elif order == 2:
        for f in Fmatrix:
            Jmatrix.append([df2(f, h, x, 0), df2(f, h, x, 1), df2(f, h, x, 2), df2(f, h, x, 3), df2(f, h, x, 4)])
        
        
    return np.array(Jmatrix, np.float64)    



## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.




In [74]:
# parcial derivatives by finite differences - FIRST ORDER
def df1(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/h
    
    return df_xk

df1(f1,0.05, x, 0)

80.0


10.999999999999943

### Método de Newton

In [84]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    k = 1
    print("Começando com Xo = ", x)
    while k <= maxIterations:
        Jx = J(Fmatrix, x, 1)
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(-F(x), np.float64))
        x = x + y
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            break
        else:
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            k += 1
        
    return output

In [87]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
methodNewton(n, Xo, tolerance, maxIterations, F)
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [10 10 10 10 10]
1  Iteração x= [0.01065436 9.99921773 5.0035254  5.03889604 0.0364584 ] Norma:  15.768211667204358
2  Iteração x= [ 0.01001145 10.01008704  2.50296252  2.59475134  0.03674464] Norma:  3.496680801986159
3  Iteração x= [9.98392590e-03 1.00835857e+01 1.24778960e+00 1.44170796e+00
 3.68865739e-02] Norma:  1.7059807076471545
4  Iteração x= [9.69190112e-03 1.04286103e+01 6.11188085e-01 9.78727969e-01
 3.69552388e-02] Norma:  0.8594498166837541
5  Iteração x= [8.52579206e-03 1.18233469e+01 2.79490585e-01 8.67730905e-01
 3.69852015e-02] Norma:  1.437927432201653
6  Iteração x= [5.50011558e-03 1.65671346e+01 1.09536521e-01 8.60070632e-01
 3.69914792e-02] Norma:  4.746838284734169
7  Iteração x= [2.98071788e-03 2.72155964e+01 6.22466538e-02 8.59733148e-01
 3.69768143e-02] Norma:  10.648567126855296
8  Iteração x= [3.45110977e-03 3.17482566e+01 6.86179925e-02 8.59520369e-01
 3.69619006e-02] Norma:  4.532664680065862
9  Iteração x= [3.43028257e-03 3.13285458e+0

### Método de Broyden

In [ ]:
Po = x # initial approximation
tolerance = .0005 # arbitrary
maxIterations = 100000 # so it doesnt loop forever in case of failure
def f(x):
    return
def df(x):
    return
def J(x):
    result = df(x)
    return result
def F(x):
    return


def methodBroyden(n, x, tolerance, maxIterations): # n is the number of equations and unknowns
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    Ao = J(x)
    v = F(x)
    A = np.transpose(Ao) #use gaussian elimination
    s = -A * v
    x = x + s
    k = 2
    
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -A* y
        p = -np.transpose(s) * z
        ut = np.transpose(np.transpose(s) * A)
        A = A + (1/p)*(s+z)*ut
        s = -A * v
        x = x + s
        if np.linalg.norm(s) < tolerance:
            output = x
            break
        
        k += 1
        
    return output
          

### Comparação dos resultados

## ITEM 3

#### Bibliografia e referências: 

R.L. Burden & J.D. Faires, Numerical Analysis, Capítulos 2 e 10